Se desarrollará un RAG que toma archivos PDF como fuente de información. El sistema segmentará el texto de los documentos en partes, las guardará en una base de datos vectorial y generará respuestas en español basadas en esos fragmentos.

Importamos las dependencias.

In [42]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

Cargamos nuestros pdf y guardamos el contenido de los tres pdf en una misma variable.

In [43]:
pdf_files = ["pdf/cristiano.pdf","pdf/messi.pdf","pdf/neymar.pdf"]  
documentos = []
for document in pdf_files:
    loader = PyPDFLoader(document) # Carga el documento
    datos = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)   
    documents = splitter.split_documents(datos)
    documentos.extend(documents)

In [44]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

Creamos nuestra base de datos vectorial con la información de los pdfs.

In [45]:
vector_store = Chroma.from_documents(
    documents=documentos,
    collection_name="pdfs",
    embedding=embeddings, 
)

In [46]:
retriever = vector_store.as_retriever()

Le damos nuestro contexto de información de pdfs a nuestro modelo.

In [48]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

llm = "llama3.2"
modelo = ChatOllama(model=llm)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | modelo
    | StrOutputParser()
)

Y Preguntamos.

In [49]:
chain.invoke("Quien es messi?")

'Lionel Andrés Messi Cuccittini, conocido como Leo Messi, es un futbolista argentino que juega como delantero o centrocampista. Es capitán de la selección de Argentina y ha jugado con el Fútbol Club Barcelona durante más de veinte años.'

In [50]:
chain.invoke("Quien es cristiano Ronaldo?")

'Cristiano Ronaldo dos Santos Aveiro es un futbolista portugués nacido en Funchal, Madeira, el 5 de febrero de 1985. Es delantero y juega actualmente para el Al-Nassr F. C. de la Liga Profesional Saudí. También es capitán de la selección de Portugal, máximo goleador histórico y jugador con más presencias en la historia de su selección (217 partidos).'

In [51]:
chain.invoke("¿Quien es Neymar Jr?")

'Neymar da Silva Santos Júnior, conocido simplemente como Neymar Jr., es un futbolista brasileño que juega como delantero en el Al-Hilal Saudi F. C. de la Liga Profesional Saudí y es internacional absoluto con la selección de Brasil. Es ampliamente reconocido como un destacado goleador y creador de juego, y ha tenido éxitos en clubes y competiciones internacionales, incluyendo cuatro títulos de liga en Francia y dos títulos de copa, lo que le valió ser votado como el Jugador del Año de la Ligue 1 en su temporada debut.'